In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.set_random_seed(777)  # for reproducibility

In [3]:
x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

In [4]:
X = tf.placeholder(tf.float32, [None, 2], name="x")
Y = tf.placeholder(tf.float32, [None, 1], name="y")

In [6]:
# TensorBoard 의 layer 분산 표기 name_scope
with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.random_normal([2, 2]), name="weight_1")
    b1 = tf.Variable(tf.random_normal([2]), name="bias_1")
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)
    # Tensor의 종류와 대입 값을 선정 (histogram, scalar)
    # 도출되는 값은 summary 모두 Board를 통해 연결 
    tf.summary.histogram("W1", W1)
    tf.summary.histogram("b1", b1)
    tf.summary.histogram("Layer1", layer1)

In [7]:
with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.random_normal([2, 1]), name="weight_2")
    b2 = tf.Variable(tf.random_normal([1]), name="bias_2")
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    tf.summary.histogram("W2", W2)
    tf.summary.histogram("b2", b2)
    tf.summary.histogram("Hypothesis", hypothesis)

In [8]:
# cost, train 수치도 Board로 분산, 집중 표현 
with tf.name_scope("Cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    tf.summary.scalar("Cost", cost)

with tf.name_scope("Train"):
    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)


In [9]:
# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [10]:
# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs --- multiline을 위한 하부 디렉토리 파일 구성
    # summary 파일들 merge
    merged_summary = tf.summary.merge_all()
    # 파일 저장 디렉토리 선정 
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    # 그래프 보여줄 것을 명령 
    writer.add_graph(sess.graph)  

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        _, summary, cost_val = sess.run(
            [train, merged_summary, cost], feed_dict={X: x_data, Y: y_data}
        )
        # Learning 하는 과정을 Board로 전송 
        # global_step 은 Learning이 진행되는 횟수 
        writer.add_summary(summary, global_step=step)

        if step % 100 == 0:
            print(step, cost_val)

    # 파일 내에서 print한 과정 
    h, p, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data}
    )
    
    print(f"\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}")

# tensorboard --logdir=./logs --- 하위 폴더의 내용을 multiline으로 showing 


0 1.1075948
100 0.69952923
200 0.65879595
300 0.62004936
400 0.5582166
500 0.47004214
600 0.34622765
700 0.23504838
800 0.16395563
900 0.12068249
1000 0.092984885
1100 0.074178934
1200 0.06076122
1300 0.05080209
1400 0.043173186
1500 0.037178177
1600 0.03236717
1700 0.028437916
1800 0.0251809
1900 0.02244639
2000 0.020125236
2100 0.018135889
2200 0.016416445
2300 0.014919011
2400 0.0136062605
2500 0.0124484375
2600 0.011421731
2700 0.010506877
2800 0.009687953
2900 0.008952001
3000 0.0082881
3100 0.00768721
3200 0.0071415994
3300 0.006644767
3400 0.006191147
3500 0.0057759574
3600 0.00539507
3700 0.005044865
3800 0.0047222683
3900 0.004424542
4000 0.004149299
4100 0.0038944539
4200 0.0036580611
4300 0.0034385817
4400 0.0032344337
4500 0.0030443668
4600 0.002867162
4700 0.0027018404
4800 0.0025473796
4900 0.0024029384
5000 0.0022677649
5100 0.0021411385
5200 0.0020223837
5300 0.00191099
5400 0.0018063873
5500 0.0017081108
5600 0.0016157107
5700 0.0015287672
5800 0.0014469654
5900 0.0013